# 00.한국어전처리패키지


- 한국어 전처리 패키지 : https://wikidocs.net/92961
- soyNLP : https://github.com/lovit/soynlp

# 새로운 텍스트전처리 과정

## 01.데이터 로드

In [ ]:
!pip install konlpy
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 453 kB 44.1 MB/s 
--2022-08-02 06:12:44--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-08-02 06:12:44--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb69fe3a86154fe1946f0ef2db9.dl.dropboxusercontent.com/cd/0/get/BqOTUs3uMDfiTZiv3ddjJbw78Bi9RZy6yuyiCKHx_hW5-oPBKP_UUz24sBDFrrdNyCV17jwXnWts8ADmSyoU-icZfVkw2VDis-IGQ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 

path = '/content/drive/MyDrive/NLP미니플젝-10조/Data/전처리된 데이터/'

train_df = pd.read_csv(path+"violence_train.csv")
test_df = pd.read_csv(path+"violence_valid.csv")

## 02.결측치 확인

In [ ]:
print(train_df['문장'].isna().any())
print(train_df['clean'].isna().any())
print(train_df['violence'].isna().any())
print(test_df['문장'].isna().any())
print(test_df['clean'].isna().any())
print(test_df['violence'].isna().any())

False
False
False
False
False
False


## 03.결측치 행 제거

In [ ]:
train_df = train_df.dropna(axis=0)
test_df = test_df.dropna(axis=0)

## 04.clean + violence 합이 1 아니면 제거 

In [ ]:
# 탐색
remove_i = train_df[train_df['clean']+train_df['violence'] != 1.0].index 
print(remove_i)

# 제거
train_df.drop(remove_i,axis='index',inplace=True)

Int64Index([], dtype='int64')


In [ ]:
# 탐색
remove_i = test_df[test_df['clean']+test_df['violence'] != 1.0].index 
print(remove_i)

# 제거
test_df.drop(remove_i,axis='index',inplace=True)

Int64Index([], dtype='int64')


## 05.텍스트전처리

- 한국어 전처리 패키지 : https://wikidocs.net/92961
- soyNLP : https://github.com/lovit/soynlp

### 한글과 띄어쓰기를 제외한 모든 부분을 제거

In [ ]:
import re

In [ ]:
a_list = []
for i in range(len(train_df)):
  text = train_df.iloc[i]['문장']
  hangul = re.compile('[^ ㄱ-ㅣ가-힣+]') # 한글과 띄어쓰기를 제외한 모든 글자
  result = hangul.sub('', text) # 한글과 띄어쓰기를 제외한 모든 부분을 제거한
  a_list.append(result)

train_df['문장'] = a_list

In [ ]:
b_list = []
for i in range(len(test_df)):
  text = test_df.iloc[i]['문장']
  hangul = re.compile('[^ ㄱ-ㅣ가-힣+]') # 한글과 띄어쓰기를 제외한 모든 글자
  result = hangul.sub('', text) # 한글과 띄어쓰기를 제외한 모든 부분을 제거한
  b_list.append(result)
  
test_df['문장'] = b_list

In [ ]:
# train_df['문장'] = train_df['문장'].str.re('[특수문자]')

### _#@#(개인정보) 포함한 행 제거

In [ ]:
index_train = train_df[train_df['문장'].str.find("#@#") != -1].index
index_test = test_df[test_df['문장'].str.find("#@#") != -1].index

train_df = train_df.drop(index = index_train)
test_df = test_df.drop(index = index_test)

### 중복된 문자열 줄이기

soyNLP를 활용한 중복된 문자열 줄이기 : 잘 안됨  
기본 코드로 중복된 문자열 줄이기 : O ( 2자리 )

In [ ]:
%%time
# 약 20초

def text_normalize(text):
    count = {}
    normalized_text = ""
    for c in text:
        if c not in count:
            count = {c: 1}
        else:
            count[c] += 1
        if count[c] <= 2:
            normalized_text += c

    return normalized_text

text_nor_list_train = []
text_nor_list_test = []

for i in range(len(train_df)):
  text = train_df.iloc[i]['문장']
  text_nor_list_train.append(text_normalize(text))

for i in range(len(test_df)):
  text = test_df.iloc[i]['문장']
  text_nor_list_test.append(text_normalize(text))

train_df['문장'] = text_nor_list_train
test_df['문장'] = text_nor_list_test

CPU times: user 8.08 s, sys: 12.8 ms, total: 8.1 s
Wall time: 15.7 s


## 06.형태소분석기

In [ ]:
from konlpy.tag import Okt
from konlpy.tag import Komoran
from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Mecab

okt = Okt()
komoran = Komoran()
kkma = Kkma()
hannanum = Hannanum()
mecab = Mecab()

In [ ]:
%%time

## "명사", "동사", "형용사"

def tagging(input_df,tag, parts):
  words_arr = []
  for i in input_df['문장']:
    text = i.strip() # 양쪽 공백제거
    #print(text)

    sentence = [] # 문장 : 단어를 문장으로 재조합하기 위해 ㅏㅅ용

    taggings = tag.pos(text)

    if text != " " or text != "":
      for i in taggings:
        if i == 'EOS' or i == ' ' : 
          continue
        word_temp = i[0] # word 
        part = i[1] # 품사
        if not (part in parts): # ['Noun','Verb','Adjective']
          continue
        sentence.append(word_temp)
      words_arr.append(' '.join(sentence)) # 단어들을 문장으로 재조합 후, 리스트에 추가
    else:
      words_arr.append("")
  return words_arr

# 1분 13초(train기준)
# Noun : 명사, Verb : 동사 , Adject : 형용사
# train_df['okt'] = tagging(train_df, okt,['Noun','Verb','Adjective']) 
# test_df['okt'] = tagging(test_df, okt,['Noun','Verb','Adjective']) 

# 40초(train기준)
# NNG :일반명사, NNP : 고유명사, NNB : 의존명사, NR:수사, NP:대명사, VV:동사, VA:형용사
# train_df['komoran'] = tagging(train_df, komoran,['NNG','NNP','NNB','NR','NP','VV','VA']) 
# test_df['komoran'] = tagging(test_df, komoran,['NNG','NNP','NNB','NR','NP','VV','VA'])

# 14분 30초(train기준)
# NNG :일반명사, NNP : 고유명사, NNB : 의존명사, NNM:단위 의존 명사 NR:수사, NP:대명사, VV:동사, VA:형용사
# train_df['kkma'] = tagging(train_df, kkma,['NNG','NNP','NNB','NNM','NR','NP','VV','VA'])
# test_df['kkma'] = tagging(test_df, kkma,['NNG','NNP','NNB','NNM','NR','NP','VV','VA'])

# 2분 30초(train기준)
# N : 명사, P : 동사+형용사+보조용언
# train_df['hannanum'] = tagging(train_df, hannanum,['N','P'])  
# test_df['hannanum'] = tagging(test_df, hannanum,['N','P'])  

# 3초?
train_df['mecab'] = tagging(train_df, mecab,['NNG','NNP','NNB','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ'])  
test_df['mecab'] = tagging(test_df, mecab,['NNG','NNP','NNB','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ'])  

CPU times: user 4.79 s, sys: 2.83 ms, total: 4.79 s
Wall time: 5.16 s


In [ ]:
train_df

,문장,clean,violence,mecab
0,여경 할당제 어떻게 생각해,1.0,0.0,여경 할당제 생각
1,남자에게 불리한게 많기는 한데 솔직히 여혐 쳐 걸린 애들은 현실에 지 몸하나 챙길 ...,0.0,1.0,남자 불리 많 여혐 애 현실 지 몸 하나 능력 없 것 많
2,말하는거 보니 주차해도 되겠다,1.0,0.0,말 거 보 주차 되
3,잘했네 ㅋㅋ 인간 기생충 하나 키울 뻔 했다,0.0,1.0,인간 기생충 하나 뻔
4,짱깨들이 만든 것중엔 제대로 된 게 없잖아,0.0,1.0,짱깨 것 중 없
...,...,...,...,...
27999,ㄴㄴ 오우예지력 상승이래 완전 아줌마 냄새 그거 한 년전에 쓰던 말인데,0.0,1.0,ㄴ ㄴ 지력 상승 이래 완전 아줌마 냄새 그거 전 쓰 말
28000,월급 받으면서 명품이나 사는 것들은 다 나사 빠진 거지,0.0,1.0,월급 받 명품 사 것 나사 거
28001,아무튼 돼지년들 ㅋㅋ꼭 사고를 쳐요 븅신들,0.0,1.0,돼지 년 사고
28002,뒤통수 삽으로 후린다음에 저 흙파서 그대로 묻고싶다,0.0,1.0,뒤통수 삽 후린 다음 흙 파 묻


### 4가지 형태소 중 가장 긴 것만 한 컬럼에 모음

In [ ]:
train_df['long'] = train_df[['okt','komoran','kkma','hannanum']].apply(lambda x : max(x[0],x[1],x[2],x[3], key=len), axis=1 )
test_df['long'] = test_df[['okt','komoran','kkma','hannanum']].apply(lambda x : max(x[0],x[1],x[2],x[3], key=len), axis=1 )

In [ ]:
train_df[train_df['okt'].str.len() < 3]

,문장,clean,violence,okt,komoran,kkma,hannanum,long
289,ㅗㅜㅑㅅㅂㅂ,0.0,1.0,,,ㅅ ㅂ ㅂ,ㅗㅜㅑㅅㅂㅂ,ㅗㅜㅑㅅㅂㅂ
369,그러니까ㅋㅋ,1.0,0.0,,,,그러니깤ㅋ,그러니깤ㅋ
370,ㅈㅈ,1.0,0.0,,,ㅈ ㅈ,ㅈㅈ,ㅈ ㅈ
635,ㅋㅋ ㅆㅃ ㅆㅅㅌㅊ,0.0,1.0,,,ㅅ ㅊ,ㅋㅋ ㅆㅃ ㅆㅅㅌㅊ,ㅋㅋ ㅆㅃ ㅆㅅㅌㅊ
733,뭐가,1.0,0.0,뭐,뭐,뭐,뭐,뭐
...,...,...,...,...,...,...,...,...
27558,ㅇㅇ 잼민 ㅎㅇ,0.0,1.0,잼민,재 민,ㅇ ㅇ 잼 민 ㅎ ㅇ,ㅇㅇ 잼민 ㅎㅇ,ㅇ ㅇ 잼 민 ㅎ ㅇ
27623,정말이야,1.0,0.0,정말,,정말,,정말
27638,왜,1.0,0.0,왜,,외,,왜
27680,여대사람대 ㅋㅋ,1.0,0.0,사람,여대 사람 대,여대 사람 대,여대사람대 ㅋㅋ,여대사람대 ㅋㅋ


### long 컬럼 길이가 2 미만 제거

- 길이 == 0(제거)  
ex) 대부분 ㅋㅋㅋ, ㅅㅂ, ㅄ 이 전처리 과정에서 사라짐  

- 길이 == 1(제거)  
ex) 뭐 뭇 머 놈 딱 뭐 키 효 왜 긐 으 <-- 알아먹기 힘듬

- 길이 == 2(생존)  
ex) 지랄, 꺼지, 민폐, 정부, 좆같, 보지, 선동 

In [ ]:
# 확인
print(train_df[train_df['okt'].str.len() < 2].size)
print(train_df[train_df['komoran'].str.len() < 2].size)
print(train_df[train_df['kkma'].str.len() < 2].size)
print(train_df[train_df['hannanum'].str.len() < 2].size)
print(train_df[train_df['long'].str.len() < 2].size)
print('-'*50)
print(test_df[test_df['okt'].str.len() < 2].size)
print(test_df[test_df['komoran'].str.len() < 2].size)
print(test_df[test_df['kkma'].str.len() < 2].size)
print(test_df[test_df['hannanum'].str.len() < 2].size)
print(test_df[test_df['long'].str.len() < 2].size)

688
3984
1648
616
296
--------------------------------------------------
144
992
384
168
64


In [ ]:
# long 컬럼 길이가 2미만 제거
train_df = train_df[train_df['long'].str.len() >= 2]
test_df = test_df[test_df['long'].str.len() >= 2]

### 형태소별로 데이터 분할

In [ ]:
# 분할
train_okt = train_df[['okt','clean','violence']]
train_komoran = train_df[['komoran','clean','violence']]
train_kkma = train_df[['kkma','clean','violence']]
train_hannanum = train_df[['hannanum','clean','violence']]
train_long = train_df[['long','clean','violence']]


# 각각 길이가 2미만 제거 
train_okt = train_okt[train_okt['okt'].str.len() >= 2]
train_komoran = train_komoran[train_komoran['komoran'].str.len() >= 2]
train_kkma = train_kkma[train_kkma['kkma'].str.len() >= 2]
train_hannanum = train_hannanum[train_hannanum['hannanum'].str.len() >= 2]
train_long = train_long[train_long['long'].str.len() >= 2]

# 혹시 모를 결측치 제거
train_okt = train_okt.dropna(axis=1)
train_komoran = train_komoran.dropna(axis=1)
train_kkma = train_kkma.dropna(axis=1)
train_hannanum = train_hannanum.dropna(axis=1)
train_long = train_long.dropna(axis=1)

# 컬럼명 변경
train_okt.columns = ['문장', 'clean', 'violence']
train_komoran.columns = ['문장', 'clean', 'violence']
train_kkma.columns = ['문장', 'clean', 'violence']
train_hannanum.columns = ['문장', 'clean', 'violence']
train_long.columns = ['문장', 'clean', 'violence']

In [ ]:
# 분할
test_okt = test_df[['okt','clean','violence']]
test_komoran = test_df[['komoran','clean','violence']]
test_kkma = test_df[['kkma','clean','violence']]
test_hannanum = test_df[['hannanum','clean','violence']]
test_long = test_df[['long','clean','violence']]

# 각각 길이가 2미만 제거 
test_okt = test_okt[test_okt['okt'].str.len() >= 2]
test_komoran = test_komoran[test_komoran['komoran'].str.len() >= 2]
test_kkma = test_kkma[test_kkma['kkma'].str.len() >= 2]
test_hannanum = test_hannanum[test_hannanum['hannanum'].str.len() >= 2]
test_long = test_long[test_long['long'].str.len() >= 2]

# 혹시 모를 결측치 제거
test_okt = test_okt.dropna(axis=1)
test_komoran = test_komoran.dropna(axis=1)
test_kkma = test_kkma.dropna(axis=1)
test_hannanum = test_hannanum.dropna(axis=1)
test_long = test_long.dropna(axis=1)

# 컬럼명 변경
test_okt.columns = ['문장', 'clean', 'violence']
test_komoran.columns = ['문장', 'clean', 'violence']
test_kkma.columns = ['문장', 'clean', 'violence']
test_hannanum.columns = ['문장', 'clean', 'violence']
test_long.columns = ['문장', 'clean', 'violence']

In [ ]:
# 분할
train_mecab = train_df[['mecab','clean','violence']]
test_mecab = test_df[['mecab','clean','violence']]

# 각각 길이가 2미만 제거 
train_mecab = train_mecab[train_mecab['mecab'].str.len() >= 2]
test_mecab = test_mecab[test_mecab['mecab'].str.len() >= 2]

# 혹시 모를 결측치 제거
train_mecab = train_mecab.dropna(axis=1)
test_mecab = test_mecab.dropna(axis=1)

# 컬럼명 변경
train_mecab.columns = ['문장', 'clean', 'violence']
test_mecab.columns = ['문장', 'clean', 'violence']

# 06.파일 저장

In [ ]:
train_okt.to_csv("train_okt.csv", index=False, encoding="utf-8-sig")
train_komoran.to_csv("train_komoran.csv", index=False, encoding="utf-8-sig")
train_kkma.to_csv("train_kkma.csv", index=False, encoding="utf-8-sig")
train_hannanum.to_csv("train_hannanum.csv", index=False, encoding="utf-8-sig")
train_long.to_csv("train_long.csv", index=False, encoding="utf-8-sig")
train_special.to_csv("train_special.csv", index=False, encoding="utf-8-sig")

In [ ]:
test_okt.to_csv("test_okt.csv", index=False, encoding="utf-8-sig")
test_komoran.to_csv("test_komoran.csv", index=False, encoding="utf-8-sig")
test_kkma.to_csv("test_kkma.csv", index=False, encoding="utf-8-sig")
test_hannanum.to_csv("test_hannanum.csv", index=False, encoding="utf-8-sig")
test_long.to_csv("test_long.csv", index=False, encoding="utf-8-sig")
test_special.to_csv("test_special.csv", index=False, encoding="utf-8-sig")

In [ ]:
# /content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/
train_okt.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/train_okt.csv", index=False, encoding="utf-8-sig")
train_komoran.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/train_komoran.csv", index=False, encoding="utf-8-sig")
train_kkma.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/train_kkma.csv", index=False, encoding="utf-8-sig")
train_hannanum.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/train_hannanum.csv", index=False, encoding="utf-8-sig")
train_long.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/train_long.csv", index=False, encoding="utf-8-sig")

In [ ]:
test_okt.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/test_okt.csv", index=False, encoding="utf-8-sig")
test_komoran.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/test_komoran.csv", index=False, encoding="utf-8-sig")
test_kkma.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/test_kkma.csv", index=False, encoding="utf-8-sig")
test_hannanum.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/test_hannanum.csv", index=False, encoding="utf-8-sig")
test_long.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/test_long.csv", index=False, encoding="utf-8-sig")

In [ ]:
train_mecab.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/train_mecab.csv", index=False, encoding="utf-8-sig")
test_mecab.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/test_mecab.csv", index=False, encoding="utf-8-sig")